In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.optimizers import Adam
import time
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sv_df = pd.read_csv("/content/drive/MyDrive/male_vector.csv")

In [ ]:
from sklearn.utils import shuffle
sv_df_ = shuffle(sv_df)
sv_df_[:10]

,0,1,2,3,4,5,6,7,8,9,...,207,208,209,210,211,212,213,214,215,0.1
15926,-2.089487,-0.028696,-1.145155,-1.677323,-1.305383,-1.066282,-1.546661,-2.411064,-4.642456,-5.149111,...,-2.641970,-2.270246,-2.605914,-2.650199,-2.873552,-4.165000,-5.497464,-3.054674,-0.804253,male_surprise
1058,-6.385075,-10.368214,-21.123024,-21.263268,-20.354662,-19.024134,-10.373834,-11.350921,-20.277830,-26.769960,...,-17.140446,-14.432220,-15.374136,-20.794533,-26.836245,-21.658260,-19.111292,-12.762404,-6.575893,male_happy
7427,-3.780228,-6.740259,-19.693115,-16.373373,-10.546636,-10.401627,-17.800575,-19.306368,-21.706797,-19.575708,...,-15.100057,-12.830367,-13.670159,-15.184175,-16.294399,-18.178087,-16.121599,-11.816730,-7.853080,male_sad
13048,-20.928175,-21.046734,-24.805775,-24.271053,-18.600689,-15.113037,-16.140858,-19.726230,-20.521677,-19.591642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,male_nervous
16850,2.799626,2.394593,-0.396768,-1.803874,-1.756790,-1.786435,-1.386494,-1.223634,-2.295876,-3.567639,...,-5.929727,-8.065002,-8.956687,-9.125858,-9.045379,-8.750189,-8.282635,-6.511889,-4.449636,male_surprise
18184,-1.994790,-2.956062,-3.549686,-3.444111,-4.053364,-5.667073,-7.641396,-8.762446,-8.679731,-9.252987,...,-5.080478,-5.657176,-5.566310,-5.532699,-5.539927,-5.889997,-6.072390,-5.040013,-3.976037,male_surprise
3669,-6.857992,-10.458368,-30.503002,-31.635286,-32.125730,-30.752989,-30.640526,-25.041683,-24.403522,-31.261170,...,-26.204052,-28.083191,-37.448402,-39.734184,-40.262356,-41.351299,-42.787758,-44.522549,-47.525826,male_sad
20440,-6.963369,-8.113847,-13.592002,-18.920856,-15.851095,-8.679333,-8.104341,-11.399268,-14.465379,-15.318566,...,-0.188873,-0.787980,-1.734234,-1.724719,-4.051534,-6.313464,-9.941919,-14.713306,-17.383924,male_neutral
15178,-1.151470,-1.146429,-3.140477,-4.056997,-7.292954,-12.014184,-17.941956,-22.356610,-26.386497,-29.139526,...,-29.395874,-30.016853,-31.623714,-35.300018,-35.318581,-36.631943,-26.125824,-13.253776,-9.787218,male_surprise
14734,-0.269018,-1.942559,-5.625400,-6.180378,-7.488385,-7.762523,-11.410010,-16.536415,-17.514463,-7.828688,...,-35.273842,-37.309921,-38.030319,-22.905903,-11.574247,-5.054296,-4.159758,-4.410606,-4.423141,male_surprise


In [ ]:
sv_df_=sv_df_.fillna(0)

In [ ]:
X = sv_df_.drop(sv_df_.iloc[:, [-1]], axis = 1)
y = sv_df_.iloc[:, [-1]]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
14686,-0.105216,-1.712758,-5.810176,-5.731721,-6.344481,-5.109494,-5.171997,-6.434077,-10.559901,-14.432652,...,-18.923372,-19.519773,-16.928682,-15.455103,-16.974333,-9.533895,-4.692722,-4.066540,-2.634818,-1.666119
14068,-14.510300,-15.645379,-19.415570,-20.473713,-21.969126,-22.731977,-18.432850,-14.248114,-13.082366,-13.526479,...,-15.052244,-17.597017,-18.782438,-18.791122,-15.459957,-11.531445,-10.211040,-12.549198,-19.542551,-20.429018
4748,-41.176346,-38.081776,-19.374390,-13.663591,-15.599773,-17.865800,-18.754017,-20.841570,-24.058516,-25.442675,...,-13.518511,-17.789637,-21.433413,-13.834156,-7.480281,-8.278307,-11.751905,-14.641155,-12.230773,-8.335691
3030,-19.344454,-21.974060,-27.460133,-32.901615,-34.961544,-34.680557,-32.973740,-34.143875,-31.892885,-28.685741,...,-22.045107,-21.301636,-22.634155,-23.341255,-24.021187,-24.602598,-25.669355,-28.230803,-32.335110,-37.783745
21568,-6.064633,-9.658596,-15.753611,-18.299303,-23.871305,-26.074211,-27.523844,-24.465115,-21.883944,-19.615498,...,-16.988924,-16.778099,-13.424494,-12.720627,-11.197775,-13.210751,-14.082678,-14.458259,-10.536268,-4.301311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15660,-10.207984,-13.262000,-20.541320,-22.552067,-20.529053,-15.395323,-13.530489,-13.682388,-14.779338,-7.228599,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21653,-6.807521,-7.928945,-13.926022,-9.820612,-7.400405,-4.633034,-5.156332,-7.463051,-7.298455,-5.288434,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11287,-6.982593,-8.491969,-17.119108,-17.296919,-17.820114,-15.514799,-15.182044,-18.192028,-16.152948,-11.947988,...,-22.522352,-25.857327,-31.767588,-36.856075,-40.598938,-43.251915,-44.412380,-31.530193,-26.756033,-27.352682
849,-11.518507,-13.454533,-19.849312,-17.455986,-14.184492,-15.463019,-17.215523,-17.143950,-19.890377,-22.111917,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)
y_val = lb.transform(y_val)

y_train =  to_categorical(y_train, num_classes=12)
y_test =  to_categorical(y_test, num_classes=12)
y_val =  to_categorical(y_val, num_classes=12)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [ ]:
# 세번째 차원을 추가
# -> CNN은 3차원 배열을 입력을 받는다.
x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)
x_valcnn= np.expand_dims(X_val, axis=2)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('male_best_model.h5', save_best_only=True)

In [ ]:
# 간단한 RNN 모델 정의 (Bidirectional LSTM 사용, 4층, 12개의 출력 클래스)
model = Sequential()
# model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(12, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
model.fit(x_traincnn, y_train,
          epochs=100,
          batch_size=32,
          validation_data=(x_valcnn, y_val),
          callbacks = model_checkpoint)

Epoch 1/100
445/445 [==============================] - 55s 92ms/step - loss: 1.0580 - accuracy: 0.5442 - val_loss: 0.9297 - val_accuracy: 0.5884
Epoch 2/100
  1/445 [..............................] - ETA: 30s - loss: 0.8289 - accuracy: 0.6562

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


445/445 [==============================] - 39s 88ms/step - loss: 0.9402 - accuracy: 0.5748 - val_loss: 0.9362 - val_accuracy: 0.5704
Epoch 3/100
445/445 [==============================] - 39s 87ms/step - loss: 0.8931 - accuracy: 0.5956 - val_loss: 0.9317 - val_accuracy: 0.5679
Epoch 4/100
445/445 [==============================] - 39s 88ms/step - loss: 0.8467 - accuracy: 0.6124 - val_loss: 0.7765 - val_accuracy: 0.6564
Epoch 5/100
445/445 [==============================] - 37s 83ms/step - loss: 0.7884 - accuracy: 0.6347 - val_loss: 0.8260 - val_accuracy: 0.6328
Epoch 6/100
445/445 [==============================] - 37s 83ms/step - loss: 0.7983 - accuracy: 0.6451 - val_loss: 0.7047 - val_accuracy: 0.6937
Epoch 7/100
445/445 [==============================] - 37s 83ms/step - loss: 0.6425 - accuracy: 0.7204 - val_loss: 0.6114 - val_accuracy: 0.7317
Epoch 8/100
445/445 [==============================] - 39s 88ms/step - loss: 0.5607 - accuracy: 0.7554 - val_loss: 0.5721 - val_accuracy: 0.74

In [ ]:
model_name = 'Emotion_Voice_Detection_Model.h5'
save_dir = os.path.join(os.getcwd(), 'saved_models')
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at /content/saved_models/Emotion_Voice_Detection_Model.h5 
